In [1]:
import requests
import time
import pandas as pd
import numpy as np
import psycopg2


In [2]:
host = "34.70.251.126"
port = 5432
user = "admin"
password = "root"
database= "umweltmonitoring"

### Messwerte der einzelnen Stationen in ein Dataframe speichern

In [3]:
#Messstationen definieren
boxes_dic = {
    "Mersch Luxembourg" : "5ae4a726223bd80019a367a6",
    "Ettlingen Deutschland" : "5dde9523ba7944001da4150e",
    "Stuttgart Deutschland" : "5c08379b1c28f9001a3f580c",
    "München Deutschland" : "5b4d11485dc1ec001b5452c7",
    "Berlin Deutschland" : "65e8d93acbf5700007f920ca",
    "Hamburg Deutschland" : "590e0b0a51d3460011c725c4",
    "Dortmund Deutschland" : "605f498077a88b001bba3dc0",
    "Jena Deutschland" : "64722d1c9be0580007f776d9"
}

In [4]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_messwerte = pd.DataFrame(columns=["time", "temperature","luftfeuchtigkeit", "pm10", "pm2_5", "senseid_fk"])

In [5]:

# Assuming df_messwerte is already defined as a DataFrame

new_data = []  # A list to collect new data
for box_id, box_value in boxes_dic.items():
    
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None

    # Traverse through the 'sensors' list
    for sensor in box.get('sensors', []):
        # Check if the sensor is for Temperature
        if sensor.get('title') == 'Temperatur':
            # Access the temperature value
            temperature_value = sensor.get('lastMeasurement', {}).get('value')
        
        if sensor.get('title') == 'PM10':
            # Access the PM10 value
            pm10_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'PM2.5':
            # Access the PM2.5 value
            pm25_value = sensor.get('lastMeasurement', {}).get('value')

        if sensor.get('title') == 'rel. Luftfeuchte':
            # Access the relative humidity value
            rel_Luftfeuchte_value = sensor.get('lastMeasurement', {}).get('value')
            
            break  # Exit the loop once the temperature is found
        
    # Append new data to the list
    new_data.append({"temperature": temperature_value, "time": time.strftime("%Y-%m-%d %H:%M:%S"),
                     "luftfeuchtigkeit": rel_Luftfeuchte_value, "pm10": pm10_value, "pm2_5": pm25_value,
                     "senseid_fk": box_value})
    
    # Convert the new data list to a DataFrame
new_df = pd.DataFrame(new_data)
    
    # Append the new DataFrame to the existing DataFrame
df_messwerte = pd.concat([df_messwerte, new_df], ignore_index=True)
df_messwerte = df_messwerte.fillna(0)

# Convert specific columns to float64, handling non-numeric values
columns_to_convert = ["temperature", "luftfeuchtigkeit", "pm10", "pm2_5"]
for column in columns_to_convert:
    df_messwerte[column] = pd.to_numeric(df_messwerte[column], errors='coerce')

df_messwerte = df_messwerte.round(2)


In [6]:
df_messwerte

,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk
0,2024-05-15 18:12:53,18.50,99.90,4.20,3.00,5ae4a726223bd80019a367a6
1,2024-05-15 18:12:54,18.32,73.25,7.80,7.10,5dde9523ba7944001da4150e
2,2024-05-15 18:12:54,23.30,55.60,2.83,2.63,5c08379b1c28f9001a3f580c
3,2024-05-15 18:12:54,17.24,69.25,10.77,3.38,5b4d11485dc1ec001b5452c7
4,2024-05-15 18:12:54,26.50,30.20,0.00,0.00,65e8d93acbf5700007f920ca
5,2024-05-15 18:12:54,28.90,37.00,4.87,1.50,590e0b0a51d3460011c725c4
6,2024-05-15 18:12:54,22.61,0.00,0.00,0.00,605f498077a88b001bba3dc0
7,2024-05-15 18:12:54,24.82,27.19,3.28,1.50,64722d1c9be0580007f776d9


In [7]:
df_messwerte["time"] = pd.to_datetime(df_messwerte["time"])

In [8]:
df_messwerte.dtypes

time                datetime64[ns]
temperature                float64
luftfeuchtigkeit           float64
pm10                       float64
pm2_5                      float64
senseid_fk                  object
dtype: object

In [9]:
import sqlalchemy

# Define your PostgreSQL connection string
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine
engine = sqlalchemy.create_engine(connection_string)

try:
    # Connect to the database using psycopg2
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Now you can proceed with inserting the DataFrame into the PostgreSQL database
    df_messwerte.to_sql("value", con=engine, if_exists="append", index=False)
    print("Daten erfolgreich in die Datenbank eingefügt.")
    
except Exception as e:
    print(f"Fehler beim Herstellen der Verbindung zur Datenbank oder Einfügen der Daten: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.
Daten erfolgreich in die Datenbank eingefügt.


### Zweites DataFrame für Daten der Messstationen erstellen

In [10]:
# Erstelle ein leeres DataFrame mit Spaltennamen
df_stations = pd.DataFrame(columns=["Ort","Land","Lat", "Lon", "Hoehe", "senseID"])

In [11]:
new_data = []  # Eine leere Liste, um neue Daten zu sammeln
for box_id, box_value in boxes_dic.items():
    try:
        response = requests.get(f'https://api.opensensemap.org/boxes/{box_value}?format=json')
        box = response.json() 
    except:
        print("FUCK YOU SENSEBOX NO WORKING TODAY" + box_value)
    temperature_value = None
    lat = None
    lon = None
    hoehe = None
    pm10_value = None
    pm25_value = None
    rel_Luftfeuchte_value = None
    ort_Land = box_id.split()

    # Extrahiere Koordinaten
    current_location = box.get('currentLocation', {})
    if current_location:
        coordinates = current_location.get('coordinates')
        if coordinates:
            lon = coordinates[0]
            lat = coordinates[1]
            height = coordinates[2] if len(coordinates) >= 3 else None

        
    # Füge neue Daten zur Liste hinzu
    new_data.append({"Ort": ort_Land[0], "Land":ort_Land[1], "Lat": lat,"Lon":lon,"Hoehe":height, "senseID": box_value})
    
    # Konvertiere die neue Datenliste in ein DataFrame
new_df = pd.DataFrame(new_data)
    
    # Hänge das neue DataFrame an das bestehende DataFrame an
df_stations = pd.concat([df_stations, new_df], ignore_index=True)

FUCK YOU SENSEBOX NO WORKING TODAY590e0b0a51d3460011c725c4


In [12]:
df_stations

,Ort,Land,Lat,Lon,Hoehe,senseID
0,Mersch,Luxembourg,49.754663,6.105682,250.0,5ae4a726223bd80019a367a6
1,Ettlingen,Deutschland,48.926209,8.374409,4.0,5dde9523ba7944001da4150e
2,Stuttgart,Deutschland,48.779610,9.213723,280.0,5c08379b1c28f9001a3f580c
3,München,Deutschland,48.146546,11.576033,20.0,5b4d11485dc1ec001b5452c7
4,Berlin,Deutschland,52.511270,13.397356,45.0,65e8d93acbf5700007f920ca
5,Hamburg,Deutschland,52.511270,13.397356,45.0,590e0b0a51d3460011c725c4
6,Dortmund,Deutschland,51.497350,7.494606,NaN,605f498077a88b001bba3dc0
7,Jena,Deutschland,50.934561,11.589701,146.0,64722d1c9be0580007f776d9
